In [1]:
import numpy as np
from torch.utils import data
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler

class Dataset(data.Dataset):
    def __init__(self, X1, Y1):
        self.X1 = X1
        self.Y1 = Y1

    def __len__(self):
        return len(self.X1)

    def __getitem__(self, index):
        x = self.X1[index]
        y1 = self.Y1[index]
        return x, y1
    
seed = 2021
torch.manual_seed(seed)
torch.cuda.manual_seed(0)
np.random.seed(seed)


def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

xTrain=np.load("../extracted_features/Change/input_features_train_grp3.npy")
yTrain=np.load("../extracted_features/Change/labels_train_grp3.npy")

xTest=np.load("../extracted_features/Change/input_features_test_grp3.npy")
yTest=np.load("../extracted_features/Change/labels_test_grp3.npy")


xVal=np.load("../extracted_features/Change/input_features_val_grp3.npy")
yVal=np.load("../extracted_features/Change/labels_val_grp3.npy")

# train_features=np.load("../extracted_features/bl/0/GRP-3/input_features_train_grp3.npy")
# labels_train=np.load("../extracted_features/bl/0/GRP-3/labels_train_grp3.npy")

# test_features=np.load("../extracted_features/bl/0/GRP-3/input_features_test_grp3.npy")
# labels_test=np.load("../extracted_features/bl/0/GRP-3/labels_test_grp3.npy")

# val_features=np.load("../extracted_features/bl/0/GRP-3/input_features_val_grp3.npy")
# labels_val=np.load("../extracted_features/bl/0/GRP-3/labels_val_grp3.npy")



In [2]:
# print(labels_train.shape,labels_train)

In [3]:
# print(train_features.shape,labels_train.shape)
# print(test_features.shape,labels_test.shape)
# print(val_features.shape,labels_val.shape)

In [4]:
print(xTrain.shape,yTrain.shape)
print(xTest.shape,yTest.shape)
print(xVal.shape,yVal.shape)

(512, 19, 8) (512,)
(122, 19, 8) (122,)
(118, 19, 8) (118,)


In [5]:
# xTrain=np.append(train_features,xTrain,axis=0)
# xTest=np.append(test_features,xTest,axis=0)
# xVal=np.append(val_features,xVal,axis=0)
# yTrain=np.append(labels_train,yTrain,axis=0)
# yTest=np.append(labels_test,yTest,axis=0)
# yVal=np.append(labels_val,yVal,axis=0)

In [6]:
import os
def readFeatures(path):
    features=np.array([])
    start=0
    for file in os.listdir(path):
        d = os.path.join(path, file)
        datafile=d+"/features.npy"
        try:
            temp=np.load(datafile)
#             print(datafile)
#             print(temp.shape)
            if(start==0):
                features=temp
                if(d[-10]=='S'):
                    label=np.ones(len(temp))
                else:
                    label=np.zeros(len(temp))
                start=1
            else:
                features=np.vstack((features,temp))
                if(d[-10]=='S'):
                    label=np.append(label,np.ones(len(temp)))
                else:
                    label=np.append(label,np.zeros(len(temp)))
        except IOError:
            print('file not in Scripted')

            
    return features,label

def load_data(path,mode):
    if mode == "test":
        xTest,yTest=readFeatures(path+"/"+mode)
#         xTest = xTest.reshape(len(xTest),1,19,8)
        return Dataset(xTest,yTest)
        
    elif mode == "train":
        xTrain,yTrain=readFeatures(path+"/"+mode)
#         xTrain = xTrain.reshape(len(xTrain),1,19,8)
        return Dataset(xTrain,yTrain)
        
    elif mode == "val":
        xVal,yVal=readFeatures(path+"/"+mode)
#         xVal = xVal.reshape(len(xVal),1,19,8)
        return Dataset(xVal,yVal)
    else:
        print("Mode not defined")
        return

train_data = load_data("../Participant_wise_physio_change",'train')
test_data = load_data("../Participant_wise_physio_change",'test')
val_data = load_data("../Participant_wise_physio_change",'val')

In [7]:
batch_size=20

# train_data = Dataset(xTrain, yTrain)
# test_data=Dataset(xTest,yTest)
# val_data=Dataset(xVal,yVal)

# train_data = Dataset(train_features, labels_train)
# test_data=Dataset(test_features,labels_test)
# val_data=Dataset(val_features,labels_val)

train_data_loader = DataLoader(train_data,worker_init_fn=seed_worker, shuffle=True, batch_size=batch_size)
val_data_loader=DataLoader(val_data,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)
test_data_loader=DataLoader(test_data,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)

In [8]:
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [9]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler



In [10]:
class Classifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.input_dim = 8
        self.hidden_dim = 128
        self.layer_dim = 2
        self.output_dim=1
        self.rnn = nn.LSTM(self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, 75)
        self.fc2 = nn.Linear(75, self.output_dim)
        self.sig=nn.Sigmoid()
        self.relu=nn.ReLU()
    
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
#         out= self.relu(out)
        out= self.fc2(out)
        out=self.sig(out)
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.cuda() for t in (h0, c0)]

In [11]:
feature_size = 24
shared_layer_size = 512
LR = 0.0001
epoch = 200
model=Classifier()
model.cuda()
iterations_per_epoch = len(train_data_loader)
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
sched = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=LR/100))

In [12]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
best_acc1 = 0
modelname=[]
truth=[]
preds=[]

for it in range(epoch+1):
    total=len(train_data_loader)*batch_size
    train_loss = 0.
    model.train()
    for minibatch in train_data_loader:
        X, Y1  = minibatch
        X=X.cuda()
        Y1=Y1.cuda()
        output = model(X.float())
        output=output.squeeze(1)
        loss = loss_func(output, Y1.float())
        Y_hat1 = torch.ge(output, 0.5).float()
        train_loss += loss.item()
        truth.extend(Y1.tolist())
        preds.extend(Y_hat1.tolist())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sched.step()
    trainacc1=f1_score(truth,preds)
    train_loss /= total
    print("EPOCH ",it)
    print('Train : Loss: {:.4f}, Train acc1 : {:.4f}'.format(train_loss,trainacc1))
    
    count=0
    val_loss= 0.
    truth=[]
    preds=[]
    total=len(val_data_loader)*batch_size
    model.eval()    
    for minibatch in val_data_loader:
        X_valid, Y1_valid  = minibatch
        X_valid=X_valid.cuda()
        Y1_valid=Y1_valid.cuda()
        output_val = model(X_valid.float())
        output_val=output_val.squeeze(1)
        loss = loss_func(output_val, Y1_valid.float())
        Y_hat1_val = torch.ge(output_val, 0.5).float()
        val_loss += loss.item()
        truth.extend(Y1_valid.tolist())
        preds.extend(Y_hat1_val.tolist())
    valacc1=f1_score(truth,preds)
    val_loss /= total
    print('Val : Loss: {:.4f}, Val acc1 : {:.4f}'.format(val_loss,valacc1))
    if valacc1 >= best_acc1:
        best_acc1 = valacc1
        best_state = model.state_dict()
        print('Best validation accuracy1 ', best_acc1)

EPOCH  0
Train : Loss: 0.0345, Train acc1 : 0.2874
Val : Loss: 0.0346, Val acc1 : 0.0339
Best validation accuracy1  0.03389830508474576
EPOCH  1
Train : Loss: 0.0342, Train acc1 : 0.1890
Val : Loss: 0.0346, Val acc1 : 0.0333
EPOCH  2
Train : Loss: 0.0341, Train acc1 : 0.2048
Val : Loss: 0.0347, Val acc1 : 0.0000
EPOCH  3
Train : Loss: 0.0337, Train acc1 : 0.2449
Val : Loss: 0.0347, Val acc1 : 0.0000
EPOCH  4
Train : Loss: 0.0333, Train acc1 : 0.2849
Val : Loss: 0.0349, Val acc1 : 0.0580
Best validation accuracy1  0.05797101449275362
EPOCH  5
Train : Loss: 0.0328, Train acc1 : 0.3077
Val : Loss: 0.0351, Val acc1 : 0.1127
Best validation accuracy1  0.11267605633802817
EPOCH  6
Train : Loss: 0.0323, Train acc1 : 0.3426
Val : Loss: 0.0362, Val acc1 : 0.2278
Best validation accuracy1  0.22784810126582278
EPOCH  7
Train : Loss: 0.0317, Train acc1 : 0.4085
Val : Loss: 0.0366, Val acc1 : 0.2278
Best validation accuracy1  0.22784810126582278
EPOCH  8
Train : Loss: 0.0313, Train acc1 : 0.3904
Va

Val : Loss: 0.0778, Val acc1 : 0.3542
EPOCH  88
Train : Loss: 0.0094, Train acc1 : 0.8252
Val : Loss: 0.0727, Val acc1 : 0.3617
EPOCH  89
Train : Loss: 0.0094, Train acc1 : 0.8258
Val : Loss: 0.0718, Val acc1 : 0.3711
EPOCH  90
Train : Loss: 0.0100, Train acc1 : 0.8120
Val : Loss: 0.0802, Val acc1 : 0.3922
EPOCH  91
Train : Loss: 0.0089, Train acc1 : 0.8231
Val : Loss: 0.0778, Val acc1 : 0.3600
EPOCH  92
Train : Loss: 0.0084, Train acc1 : 0.8243
Val : Loss: 0.0758, Val acc1 : 0.3596
EPOCH  93
Train : Loss: 0.0075, Train acc1 : 0.8516
Val : Loss: 0.0753, Val acc1 : 0.4124
Best validation accuracy1  0.4123711340206186
EPOCH  94
Train : Loss: 0.0095, Train acc1 : 0.8357
Val : Loss: 0.0771, Val acc1 : 0.3218
EPOCH  95
Train : Loss: 0.0077, Train acc1 : 0.8417
Val : Loss: 0.0756, Val acc1 : 0.4167
Best validation accuracy1  0.4166666666666667
EPOCH  96
Train : Loss: 0.0087, Train acc1 : 0.8445
Val : Loss: 0.0788, Val acc1 : 0.3617
EPOCH  97
Train : Loss: 0.0075, Train acc1 : 0.8455
Val : Lo

EPOCH  177
Train : Loss: 0.0001, Train acc1 : 0.8977
Val : Loss: 0.1417, Val acc1 : 0.4158
EPOCH  178
Train : Loss: 0.0001, Train acc1 : 0.8977
Val : Loss: 0.1431, Val acc1 : 0.4158
EPOCH  179
Train : Loss: 0.0001, Train acc1 : 0.8977
Val : Loss: 0.1438, Val acc1 : 0.4158
EPOCH  180
Train : Loss: 0.0001, Train acc1 : 0.8977
Val : Loss: 0.1454, Val acc1 : 0.4158
EPOCH  181
Train : Loss: 0.0001, Train acc1 : 0.8977
Val : Loss: 0.1441, Val acc1 : 0.4158
EPOCH  182
Train : Loss: 0.0001, Train acc1 : 0.8977
Val : Loss: 0.1463, Val acc1 : 0.4158
EPOCH  183
Train : Loss: 0.0001, Train acc1 : 0.8977
Val : Loss: 0.1458, Val acc1 : 0.4158
EPOCH  184
Train : Loss: 0.0001, Train acc1 : 0.8977
Val : Loss: 0.1474, Val acc1 : 0.4314
EPOCH  185
Train : Loss: 0.0001, Train acc1 : 0.8997
Val : Loss: 0.1486, Val acc1 : 0.4158
EPOCH  186
Train : Loss: 0.0001, Train acc1 : 0.8977
Val : Loss: 0.1469, Val acc1 : 0.4158
EPOCH  187
Train : Loss: 0.0001, Train acc1 : 0.8977
Val : Loss: 0.1485, Val acc1 : 0.4158

In [13]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

modeltest=Classifier()
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.55 0.53 0.55 0.52 0.55 0.58


In [14]:
testdataset = load_data("../Participant_wise_physio_change1",'test')

# xTest = xTest.reshape(len(xTest),1,19,24)

# testdataset = Dataset(xTest,yTest)

test_data_loader = DataLoader(testdataset,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)

modeltest=Classifier()
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.53 0.52 0.52 0.52 0.53 0.53


In [15]:
testdataset = load_data("../Participant_wise_physio_change2",'test')

# xTest = xTest.reshape(len(xTest),1,19,24)

# testdataset = Dataset(xTest,yTest)

test_data_loader = DataLoader(testdataset,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)

modeltest=Classifier()
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.58 0.55 0.58 0.52 0.58 0.63
